In [1]:
# Recurrent Neural Network with any number of layers
#
# Works in my experiments, but still needs documenting and further testing

import numpy as np

class NeuralNetworkLayer(object):
    '''
    A Neural Network Layer
    '''
    def __init__(self, input_size, output_size, name=None):
        self.inputs_per_neuron = input_size
        self.number_of_neurons = output_size
        self.synaptic_weights = 2 * np.random.random((input_size, output_size)) - 1
        self.name = name
    
    def __repr__(self):
        return str({
            'number_of_neurons': self.number_of_neurons, 
            'inputs_per_neuron': self.inputs_per_neuron, 
            'synaptic_weights': self.synaptic_weights, 
            'name': self.name})

class RecurrentNeuralNetwork(object):
    
    def __init__(self, layer_sizes, sequence_length, name=None):
        self.layers = [ NeuralNetworkLayer(input_size, output_size, name='layer_{}'.format(i)) 
                       for (i, (input_size, output_size)) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])) ]
        # Synaptic weights for intertemporal propagation at hidden layers
        self.recurrent_layers = [ NeuralNetworkLayer(size, size, name='recurrent_layer_{}'.format(i)) 
                       for (i, size) in enumerate(layer_sizes[1:-1]) ]
        # Output storage for intertemporal propagation at hidden layers
        self.recurrent_layers_values = [ np.zeros(size) 
                       for size in layer_sizes[1:-1] ]
        self.sequence_length = sequence_length
        self.name = name
        self.__layer_sizes = layer_sizes
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def predict_t(self, inputs, previous_h_outputs_stack):
        '''
        Returns the outputs of all layers. Note that:
            - Outputs of last layer is the actual prediction
            - Outputs of the other layers are still useful (and necessary when training)
        '''
        layer_outputs_stack = []
        layer_inputs = inputs # layer_inputs for first layer
        for (layer, recurrent_layer, previous_h_outputs) in zip(
            self.layers, 
            self.recurrent_layers + [None], 
            previous_h_outputs_stack + [None]):
            # 
            if(recurrent_layer == None):
                layer_outputs = self.__sigmoid(np.dot(layer_inputs, layer.synaptic_weights))
            else:
                layer_outputs = self.__sigmoid(
                    np.dot(layer_inputs, layer.synaptic_weights) + np.dot(previous_h_outputs, recurrent_layer.synaptic_weights)
                )
            layer_outputs_stack.append(layer_outputs)
            layer_inputs = layer_outputs # layer_inputs for next layer
        return layer_outputs_stack
    
    def predict(self, inputs_sequence):
        '''
        Returns the outputs of all layers. Note that:
            - Outputs of last layer is the actual prediction
            - Outputs of the other layers are still useful (and necessary when training)
        '''
        layer_outputs_stack_sequence = [] # We will return a time sequence of all layer outputs
        # Initialize the previous hidden values for first iteration
        previous_h_outputs_stack = [ np.zeros(size) for size in self.__layer_sizes[1:-1] ]
        for inputs in inputs_sequence:
            # Getting the ouptputs and adding them to the sequence
            layer_outputs_stack = self.predict_t(inputs, previous_h_outputs_stack)
            layer_outputs_stack_sequence.append(layer_outputs_stack) 
            # Now preparing hidden values for next iteration 
            previous_h_outputs_stack = layer_outputs_stack[:-1]
        return layer_outputs_stack_sequence
    
    def train_step(self, training_set_inputs_sequence, training_set_outputs_sequence, learning_rate=0.1):
        '''
        Training function
        '''
        layer_outputs_stack_sequence = self.predict(training_set_inputs_sequence)
        # Future deltas stack for first iteration
        future_layer_deltas_stack = [ np.zeros(layer.number_of_neurons) for layer in self.recurrent_layers ] + [None]
        past_layer_outputs_stack_sequence = [[None]*(len(self.__layer_sizes) - 1)] + layer_outputs_stack_sequence[:-1]
        for (training_set_inputs, training_set_outputs, layer_outputs_stack, past_layer_outputs_stack) in reversed(list(zip(
            training_set_inputs_sequence, 
            training_set_outputs_sequence, 
            layer_outputs_stack_sequence, 
            past_layer_outputs_stack_sequence
        ))):
            #
            last_layer_outputs = layer_outputs_stack[-1]
            # Now, for each layer, we calculate errors and backpropagate
            # Calculate the error for the layer in the stack
            layer_errors_1st_comp = training_set_outputs - last_layer_outputs # layer_errors for last layer
            reversed_layer_deltas_stack = []
            for (layer_outputs, layer_inputs, layer, recurrent_layer, future_deltas, past_layer_outputs) in reversed(list(zip(
                layer_outputs_stack, 
                [training_set_inputs] + layer_outputs_stack[:-1], 
                self.layers, 
                self.recurrent_layers + [None], 
                future_layer_deltas_stack,
                past_layer_outputs_stack
            ))):
                # Calculate the deltas for this layer and recurrent_layer
                if(recurrent_layer == None):
                    layer_deltas = layer_errors_1st_comp * self.__sigmoid_derivative(layer_outputs)
                else:
                    layer_errors_2nd_comp = future_deltas.dot(recurrent_layer.synaptic_weights.T)
                    layer_deltas = (layer_errors_1st_comp + layer_errors_2nd_comp) * self.__sigmoid_derivative(layer_outputs)
                                                
                # Calculate the weight adjustments for this layer and recurrent_layer
                layer_adjustments = learning_rate * layer_inputs.T.dot(layer_deltas)
                if(recurrent_layer != None and past_layer_outputs != None):
                    recurrent_layer_adjustments = learning_rate * past_layer_outputs.T.dot(layer_deltas)
                
                # Calculate the 1st component errors for the next layer in the stack 
                layer_errors_1st_comp = layer_deltas.dot(layer.synaptic_weights.T)
                
                # Adjust the weights for this layer and recurrent_layer
                layer.synaptic_weights += layer_adjustments
                if(recurrent_layer != None and past_layer_outputs != None):
                    recurrent_layer.synaptic_weights += recurrent_layer_adjustments
                
                # Append this layer_deltas to the reversed stack, as it will be needed when exiting the loop
                reversed_layer_deltas_stack.append(layer_deltas)
            
            # Future deltas stack for next iteration
            future_layer_deltas_stack = list(reversed(reversed_layer_deltas_stack))
            
        # Returning the sequence of last layer's outputs, 
        # so it can be used to calculate loss or accuracy at each training step
        return [ layer_outputs_stack[-1] for layer_outputs_stack in layer_outputs_stack_sequence ]
        
    def __repr__(self):
        return str({'layers': self.layers, 
                    'recurrent_layers': self.recurrent_layers, 
                    'name': self.name})
    

In [2]:
# Here we test the code above by creating an training a RecurrentNeuralNetwork
# To do this we are creating and training the RNN to learn to sum two binary numbers, which is exactly the same example as
# was done in https://github.com/llSourcell/recurrent_neural_net_demo (kudos for Siraj Raval for a really good demo)
# We will be using 3 layers (1 input, 1 hidden and 1 output), so just one hidden layer. Note that we don't need more
# than one hidden layer to solve this problem, anyway, we will test a model with more than 1 hidden layer later on
# As we will see during training, the RNN gives really bad predictions at first, but it eventually gets quite accurate

# We will use binary inputs of 8 bits
bin_dim = 8

# This will be the source of training data
def batch_int(max_value):
    x1 = np.random.randint(max_value/2)
    x2 = np.random.randint(max_value/2)
    return [x1, x2]

# int2bin mapping (better having the mapping stored than calculating same values thousands of times)
int_to_binarray = dict([ (i, b) for (i, b) in 
                        enumerate( np.unpackbits(np.array([range(2**bin_dim)], dtype=np.uint8).T, axis=1) ) ])

def binseq_to_int(binseq):
    intvalue = 0
    for (i, b) in enumerate(reversed(binseq)):
        intvalue += b * (2 ** i)
    return intvalue

# We create the model
np.random.seed(0)
model = RecurrentNeuralNetwork([2, 2*bin_dim, 1], bin_dim)

# Now training
epochs = 10000
learning_rate = 0.25
for epoch in range(epochs):
    (x1_int, x2_int) = batch_int(2**bin_dim)
    y_int = x1_int + x2_int
    (x1_bin_seq, x2_bin_seq, y_bin_seq) = ( int_to_binarray[x1_int], int_to_binarray[x2_int], int_to_binarray[y_int] )
    reversed_x_bin_seq = np.array([ [[x1_bin, x2_bin]] 
                                   for (x1_bin, x2_bin) in zip(
                                       list(reversed(x1_bin_seq)), 
                                       list(reversed(x2_bin_seq))) 
                                  ])
    reversed_y_bin_seq = np.array(list(reversed(y_bin_seq)))
    reversed_y_bin_seq = np.array( [ [[y_bin]] for y_bin in reversed(y_bin_seq)] )
    reversed_pred_bin_seq = model.train_step(
        reversed_x_bin_seq, 
        reversed_y_bin_seq, 
        learning_rate=learning_rate
    )
    pred_bin_seq = list( b[0][0] for b in reversed(reversed_pred_bin_seq))
    pred_int = binseq_to_int([ np.round(b) for b in pred_bin_seq ])
    error = sum( [ np.abs(y - p) for (y, p) in zip(y_bin_seq, pred_bin_seq) ] )
    print('=================================')
    print('Epoch #{}: error={}'.format(epoch, error))
    print('{} + {} = {} : {}'.format(x1_int, x2_int, y_int, pred_int))
    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:122: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Epoch #0: error=3.4563866323576504
9 + 60 = 69 : 1.0
Epoch #1: error=3.952306014598911
91 + 111 = 202 : 0.0
Epoch #2: error=4.3425462729076365
35 + 83 = 118 : 33.0
Epoch #3: error=3.8068751589958603
76 + 18 = 94 : 237.0
Epoch #4: error=3.531505593061305
113 + 74 = 187 : 253.0
Epoch #5: error=3.950765365810254
98 + 97 = 195 : 255.0
Epoch #6: error=3.8794065389392243
43 + 3 = 46 : 255.0
Epoch #7: error=4.249013720243548
12 + 100 = 112 : 237.0
Epoch #8: error=3.80619161696441
58 + 1 = 59 : 10.0
Epoch #9: error=3.597049146754757
113 + 0 = 113 : 253.0
Epoch #10: error=3.684606004715125
39 + 24 = 63 : 53.0
Epoch #11: error=3.6098562775568785
58 + 36 = 94 : 255.0
Epoch #12: error=4.282292873871351
99 + 69 = 168 : 255.0
Epoch #13: error=4.036969234004784
6 + 3 = 9 : 6.0
Epoch #14: error=4.600963318398451
98 + 121 = 219 : 0.0
Epoch #15: error=4.082929049871744
40 + 60 = 100 : 253.0
Epoch #16: error=3.665747997852952
33 + 28 = 61 : 33.0
Epoch #17: error=3.7797061214158547
68 + 26 = 94 : 125.0
Ep

46 + 118 = 164 : 0.0
Epoch #153: error=3.5408949219462498
32 + 34 = 66 : 0.0
Epoch #154: error=3.924350746516687
115 + 87 = 202 : 0.0
Epoch #155: error=4.121026922559558
124 + 25 = 149 : 0.0
Epoch #156: error=3.6468028980317513
46 + 114 = 160 : 0.0
Epoch #157: error=4.737839173635296
107 + 52 = 159 : 0.0
Epoch #158: error=3.946254467893423
105 + 110 = 215 : 252.0
Epoch #159: error=4.08394476257178
64 + 76 = 140 : 255.0
Epoch #160: error=3.9324550744464832
118 + 73 = 191 : 254.0
Epoch #161: error=4.3027404953939765
18 + 4 = 22 : 255.0
Epoch #162: error=4.276180334853593
7 + 3 = 10 : 255.0
Epoch #163: error=4.032254810098139
48 + 76 = 124 : 0.0
Epoch #164: error=4.056952926136979
37 + 116 = 153 : 252.0
Epoch #165: error=4.024491419477413
109 + 48 = 157 : 108.0
Epoch #166: error=3.584296639412867
109 + 82 = 191 : 253.0
Epoch #167: error=4.401424466444741
51 + 49 = 100 : 255.0
Epoch #168: error=4.18932383308073
71 + 90 = 161 : 255.0
Epoch #169: error=4.1681703994408625
49 + 110 = 159 : 0.0

Epoch #293: error=3.9110047862843205
19 + 107 = 126 : 223.0
Epoch #294: error=3.962739566206791
8 + 96 = 104 : 255.0
Epoch #295: error=4.093561860142176
81 + 69 = 150 : 221.0
Epoch #296: error=3.858916275983534
124 + 100 = 224 : 252.0
Epoch #297: error=4.200223917675756
1 + 125 = 126 : 0.0
Epoch #298: error=4.535534312965872
29 + 5 = 34 : 255.0
Epoch #299: error=4.144044645353495
80 + 102 = 182 : 0.0
Epoch #300: error=4.0288896436779655
120 + 40 = 160 : 252.0
Epoch #301: error=3.5916876220686236
22 + 30 = 52 : 0.0
Epoch #302: error=3.888778504516681
127 + 19 = 146 : 0.0
Epoch #303: error=3.35162999513668
4 + 20 = 24 : 0.0
Epoch #304: error=3.8118966195813253
106 + 74 = 180 : 0.0
Epoch #305: error=4.210856429207062
123 + 67 = 190 : 0.0
Epoch #306: error=4.177537760710259
86 + 60 = 146 : 254.0
Epoch #307: error=4.123382871111855
101 + 122 = 223 : 244.0
Epoch #308: error=4.448883527187501
93 + 41 = 134 : 255.0
Epoch #309: error=3.8200252724490484
40 + 60 = 100 : 252.0
Epoch #310: error=3.

101 + 31 = 132 : 253.0
Epoch #436: error=3.760991697945805
79 + 83 = 162 : 0.0
Epoch #437: error=3.529038032347459
15 + 51 = 66 : 0.0
Epoch #438: error=3.630324810853924
121 + 12 = 133 : 0.0
Epoch #439: error=4.209246691752755
45 + 10 = 55 : 0.0
Epoch #440: error=4.103294939850427
116 + 105 = 221 : 16.0
Epoch #441: error=4.070134619043284
80 + 70 = 150 : 255.0
Epoch #442: error=3.962802119727307
21 + 94 = 115 : 253.0
Epoch #443: error=3.981858756636192
67 + 80 = 147 : 255.0
Epoch #444: error=4.051990190195907
64 + 1 = 65 : 255.0
Epoch #445: error=3.826509188921576
50 + 96 = 146 : 0.0
Epoch #446: error=4.181973003048888
107 + 123 = 230 : 0.0
Epoch #447: error=3.9113948832000633
82 + 82 = 164 : 165.0
Epoch #448: error=4.086786762912877
57 + 22 = 79 : 0.0
Epoch #449: error=3.9288201444792774
15 + 15 = 30 : 159.0
Epoch #450: error=4.184311846775554
28 + 71 = 99 : 156.0
Epoch #451: error=3.9252119882414673
27 + 88 = 115 : 185.0
Epoch #452: error=3.8016552219067723
57 + 58 = 115 : 253.0
Epoc

...

123 + 25 = 148 : 148.0
Epoch #9293: error=0.2298716140337729
28 + 83 = 111 : 111.0
Epoch #9294: error=0.2683313971622308
67 + 95 = 162 : 162.0
Epoch #9295: error=0.2283068206318079
86 + 109 = 195 : 195.0
Epoch #9296: error=0.1204491654798328
93 + 96 = 189 : 189.0
Epoch #9297: error=0.12164375981575168
96 + 38 = 134 : 134.0
Epoch #9298: error=0.19553585531740675
29 + 50 = 79 : 79.0
Epoch #9299: error=0.2558190246967099
111 + 30 = 141 : 141.0
Epoch #9300: error=0.21194717536377278
105 + 14 = 119 : 119.0
Epoch #9301: error=0.21584673604873617
103 + 37 = 140 : 140.0
Epoch #9302: error=0.09380096676912977
4 + 105 = 109 : 109.0
Epoch #9303: error=0.1841098700441823
59 + 20 = 79 : 79.0
Epoch #9304: error=0.14389353178357095
23 + 16 = 39 : 39.0
Epoch #9305: error=0.1595333553572959
52 + 22 = 74 : 74.0
Epoch #9306: error=0.08228358624224388
6 + 24 = 30 : 30.0
Epoch #9307: error=0.20216730511409142
27 + 87 = 114 : 114.0
Epoch #9308: error=0.18929574403027882
119 + 42 = 161 : 161.0
Epoch #9309: e

90 + 13 = 103 : 103.0
Epoch #9439: error=0.2652216555245947
79 + 126 = 205 : 205.0
Epoch #9440: error=0.1758357507472661
30 + 79 = 109 : 109.0
Epoch #9441: error=0.08433205158985799
113 + 12 = 125 : 125.0
Epoch #9442: error=0.24165026609475646
127 + 93 = 220 : 220.0
Epoch #9443: error=0.15086182890427913
42 + 49 = 91 : 91.0
Epoch #9444: error=0.13741489694588682
14 + 3 = 17 : 17.0
Epoch #9445: error=0.16270267258059762
58 + 49 = 107 : 107.0
Epoch #9446: error=0.2171950314138839
10 + 57 = 67 : 67.0
Epoch #9447: error=0.16935762568629342
56 + 122 = 178 : 178.0
Epoch #9448: error=0.15980810585441396
80 + 67 = 147 : 147.0
Epoch #9449: error=0.18414010268705674
13 + 7 = 20 : 20.0
Epoch #9450: error=0.2154308755957221
115 + 82 = 197 : 197.0
Epoch #9451: error=0.18700937435512485
111 + 103 = 214 : 214.0
Epoch #9452: error=0.12089304883176957
16 + 29 = 45 : 45.0
Epoch #9453: error=0.16316513007629047
70 + 97 = 167 : 167.0
Epoch #9454: error=0.23090145928998865
53 + 94 = 147 : 147.0
Epoch #9455

Epoch #9592: error=0.15868495295459825
28 + 78 = 106 : 106.0
Epoch #9593: error=0.25678112755640825
78 + 123 = 201 : 201.0
Epoch #9594: error=0.21017885625495417
84 + 46 = 130 : 130.0
Epoch #9595: error=0.12065120265605747
38 + 67 = 105 : 105.0
Epoch #9596: error=0.18405041726840207
15 + 23 = 38 : 38.0
Epoch #9597: error=0.22774394776550083
100 + 126 = 226 : 226.0
Epoch #9598: error=0.1838330594127167
61 + 88 = 149 : 149.0
Epoch #9599: error=0.2302883555391318
81 + 94 = 175 : 175.0
Epoch #9600: error=0.15521875624094897
81 + 17 = 98 : 98.0
Epoch #9601: error=0.27989928298564937
58 + 38 = 96 : 96.0
Epoch #9602: error=0.15091046760163634
51 + 80 = 131 : 131.0
Epoch #9603: error=0.12534433053989968
47 + 10 = 57 : 57.0
Epoch #9604: error=0.15136612603725694
5 + 66 = 71 : 71.0
Epoch #9605: error=0.20642115535981123
14 + 101 = 115 : 115.0
Epoch #9606: error=0.09195915159225315
52 + 64 = 116 : 116.0
Epoch #9607: error=0.13678208594135224
29 + 20 = 49 : 49.0
Epoch #9608: error=0.16685458814245

Epoch #9749: error=0.16706000006519459
24 + 105 = 129 : 129.0
Epoch #9750: error=0.22913904755964687
118 + 82 = 200 : 200.0
Epoch #9751: error=0.22392220702874865
38 + 108 = 146 : 146.0
Epoch #9752: error=0.08649751260890931
11 + 0 = 11 : 11.0
Epoch #9753: error=0.14154729405861216
49 + 109 = 158 : 158.0
Epoch #9754: error=0.12632497419847383
113 + 32 = 145 : 145.0
Epoch #9755: error=0.18478553841930911
94 + 5 = 99 : 99.0
Epoch #9756: error=0.21373472584837097
125 + 114 = 239 : 239.0
Epoch #9757: error=0.20736794138081302
125 + 54 = 179 : 179.0
Epoch #9758: error=0.0876810837752951
0 + 58 = 58 : 58.0
Epoch #9759: error=0.1734411351069897
27 + 51 = 78 : 78.0
Epoch #9760: error=0.2164091015182965
106 + 122 = 228 : 228.0
Epoch #9761: error=0.28191770523654913
62 + 110 = 172 : 172.0
Epoch #9762: error=0.2520518789658478
61 + 107 = 168 : 168.0
Epoch #9763: error=0.22283105119918195
94 + 76 = 170 : 170.0
Epoch #9764: error=0.18075747156099942
59 + 38 = 97 : 97.0
Epoch #9765: error=0.20830801

Epoch #9907: error=0.08390861253577533
41 + 68 = 109 : 109.0
Epoch #9908: error=0.20978915595360564
107 + 74 = 181 : 181.0
Epoch #9909: error=0.15840559316990166
115 + 99 = 214 : 214.0
Epoch #9910: error=0.18783035741667659
31 + 72 = 103 : 103.0
Epoch #9911: error=0.14656331180161547
38 + 19 = 57 : 57.0
Epoch #9912: error=0.22127338714267691
16 + 83 = 99 : 99.0
Epoch #9913: error=0.18397438942024968
99 + 15 = 114 : 114.0
Epoch #9914: error=0.1929051258577606
64 + 111 = 175 : 175.0
Epoch #9915: error=0.20470343585627987
60 + 70 = 130 : 130.0
Epoch #9916: error=0.2720680871309406
14 + 123 = 137 : 137.0
Epoch #9917: error=0.17759455515397035
38 + 18 = 56 : 56.0
Epoch #9918: error=0.21162947086270698
52 + 94 = 146 : 146.0
Epoch #9919: error=0.16881747679660208
25 + 91 = 116 : 116.0
Epoch #9920: error=0.15031306877888684
41 + 108 = 149 : 149.0
Epoch #9921: error=0.2718967107069295
58 + 38 = 96 : 96.0
Epoch #9922: error=0.16297175494311333
123 + 48 = 171 : 171.0
Epoch #9923: error=0.22918776

In [3]:
# Now we will create a RecurrentNeuralNetwork with 2 hidden layers. Note that this is not necessary to solve the
# sum of binary numbers, but we are just doing it just to test that RecurrentNeuralNetwork class also works well with multiple
# hidden layers.
# As we will see during training, the RNN gives really bad predictions at first, but it eventually gets quite accurate

# We create the model
np.random.seed(0)
model = RecurrentNeuralNetwork([2, 2*bin_dim, 3*bin_dim, 1], bin_dim)

# Now training
epochs = 10000
learning_rate = 0.25
for epoch in range(epochs):
    (x1_int, x2_int) = batch_int(2**bin_dim)
    y_int = x1_int + x2_int
    (x1_bin_seq, x2_bin_seq, y_bin_seq) = ( int_to_binarray[x1_int], int_to_binarray[x2_int], int_to_binarray[y_int] )
    reversed_x_bin_seq = np.array([ [[x1_bin, x2_bin]] 
                                   for (x1_bin, x2_bin) in zip(
                                       list(reversed(x1_bin_seq)), 
                                       list(reversed(x2_bin_seq))) 
                                  ])
    reversed_y_bin_seq = np.array(list(reversed(y_bin_seq)))
    reversed_y_bin_seq = np.array( [ [[y_bin]] for y_bin in reversed(y_bin_seq)] )
    reversed_pred_bin_seq = model.train_step(
        reversed_x_bin_seq, 
        reversed_y_bin_seq, 
        learning_rate=learning_rate
    )
    pred_bin_seq = list( b[0][0] for b in reversed(reversed_pred_bin_seq))
    pred_int = binseq_to_int([ np.round(b) for b in pred_bin_seq ])
    error = sum( [ np.abs(y - p) for (y, p) in zip(y_bin_seq, pred_bin_seq) ] )
    print('=================================')
    print('Epoch #{}: error={}'.format(epoch, error))
    print('{} + {} = {} : {}'.format(x1_int, x2_int, y_int, pred_int))
    

Epoch #0: error=3.739105948704708
107 + 94 = 201 : 255.0
Epoch #1: error=4.022856686101072
86 + 0 = 86 : 3.0
Epoch #2: error=3.902330050625761
81 + 58 = 139 : 209.0

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:122: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.



Epoch #3: error=3.9868396661730148
12 + 91 = 103 : 1.0
Epoch #4: error=3.9964298569144576
43 + 54 = 97 : 255.0
Epoch #5: error=4.418644144258065
125 + 47 = 172 : 1.0
Epoch #6: error=3.9089775167139136
22 + 31 = 53 : 243.0
Epoch #7: error=4.22063900537954
116 + 34 = 150 : 81.0
Epoch #8: error=4.351397629498476
98 + 42 = 140 : 243.0
Epoch #9: error=4.8160281960215725
68 + 122 = 190 : 1.0
Epoch #10: error=4.477870415132559
62 + 36 = 98 : 255.0
Epoch #11: error=3.9349646103755638
83 + 44 = 127 : 211.0
Epoch #12: error=4.499143479051794
81 + 112 = 193 : 255.0
Epoch #13: error=3.833354139827843
111 + 70 = 181 : 255.0
Epoch #14: error=4.089914809220851
6 + 48 = 54 : 255.0
Epoch #15: error=4.0345574726612785
9 + 28 = 37 : 235.0
Epoch #16: error=3.6267625943938624
77 + 56 = 133 : 1.0
Epoch #17: error=4.5486981704254665
9 + 85 = 94 : 0.0
Epoch #18: error=3.8939571437978233
66 + 9 = 75 : 255.0
Epoch #19: error=3.9574350684854167
110 + 120 = 230 : 239.0
Epoch #20: error=4.634714957534192
36 + 28 

Epoch #99: error=4.195670766977552
121 + 106 = 227 : 0.0
Epoch #100: error=3.8549180993090744
3 + 116 = 119 : 255.0
Epoch #101: error=4.024043063543401
84 + 113 = 197 : 255.0
Epoch #102: error=4.073571427848601
83 + 121 = 204 : 255.0
Epoch #103: error=4.067301528879435
37 + 33 = 70 : 255.0
Epoch #104: error=3.7842670252276545
60 + 108 = 168 : 0.0
Epoch #105: error=3.5306226972570856
89 + 7 = 96 : 0.0
Epoch #106: error=3.681509666314917
60 + 37 = 97 : 0.0
Epoch #107: error=4.343587840714092
84 + 122 = 206 : 0.0
Epoch #108: error=3.9082119042545074
98 + 16 = 114 : 234.0
Epoch #109: error=4.005995732772944
45 + 104 = 149 : 253.0
Epoch #110: error=4.018382014142039
39 + 33 = 72 : 110.0
Epoch #111: error=3.6131194350670968
97 + 65 = 162 : 0.0
Epoch #112: error=3.3588991281152745
73 + 87 = 160 : 0.0
Epoch #113: error=3.9414643220422594
68 + 24 = 92 : 0.0
Epoch #114: error=4.272266289825305
6 + 97 = 103 : 0.0
Epoch #115: error=3.8916481884095724
114 + 127 = 241 : 254.0
Epoch #116: error=4.121

Epoch #207: error=4.021696311949773
30 + 22 = 52 : 254.0
Epoch #208: error=3.9218643084590172
123 + 75 = 198 : 0.0
Epoch #209: error=4.1256577487304105
35 + 84 = 119 : 34.0
Epoch #210: error=4.442908696093818
68 + 63 = 131 : 255.0
Epoch #211: error=4.099541929347104
4 + 49 = 53 : 254.0
Epoch #212: error=4.122981984145943
116 + 29 = 145 : 252.0
Epoch #213: error=4.032355491438691
63 + 61 = 124 : 0.0
Epoch #214: error=3.920524963085841
91 + 111 = 202 : 254.0
Epoch #215: error=3.9616181214514525
83 + 104 = 187 : 254.0
Epoch #216: error=3.8825297540073933
90 + 26 = 116 : 255.0
Epoch #217: error=3.7530047750811764
73 + 50 = 123 : 255.0
Epoch #218: error=3.332555671636317
95 + 94 = 189 : 255.0
Epoch #219: error=4.348456319974129
17 + 71 = 88 : 255.0
Epoch #220: error=3.9971379726899308
56 + 115 = 171 : 254.0
Epoch #221: error=3.9806824835354915
35 + 73 = 108 : 255.0
Epoch #222: error=4.148219776081087
92 + 47 = 139 : 254.0
Epoch #223: error=4.051176185965335
48 + 119 = 167 : 254.0
Epoch #224

...

Epoch #9416: error=0.057164987290315834
37 + 76 = 113 : 113.0
Epoch #9417: error=0.08052425812178685
78 + 73 = 151 : 151.0
Epoch #9418: error=0.08294787562978903
103 + 27 = 130 : 130.0
Epoch #9419: error=0.0747072173231017
101 + 94 = 195 : 195.0
Epoch #9420: error=0.06145930333135947
19 + 100 = 119 : 119.0
Epoch #9421: error=0.05910095732441343
21 + 6 = 27 : 27.0
Epoch #9422: error=0.05566911871789197
99 + 66 = 165 : 165.0
Epoch #9423: error=0.07512692443236607
117 + 38 = 155 : 155.0
Epoch #9424: error=0.050480251961546777
19 + 4 = 23 : 23.0
Epoch #9425: error=0.0718463013404419
80 + 81 = 161 : 161.0
Epoch #9426: error=0.08035569967209144
38 + 77 = 115 : 115.0
Epoch #9427: error=0.04563376449700519
126 + 107 = 233 : 233.0
Epoch #9428: error=0.08454430535823362
46 + 37 = 83 : 83.0
Epoch #9429: error=0.055428073898094773
25 + 13 = 38 : 38.0
Epoch #9430: error=0.0653535181436926
85 + 60 = 145 : 145.0
Epoch #9431: error=0.07119714525322682
37 + 42 = 79 : 79.0
Epoch #9432: error=0.048040310

Epoch #9586: error=0.08758997476556399
58 + 22 = 80 : 80.0
Epoch #9587: error=0.05294690760460739
16 + 84 = 100 : 100.0
Epoch #9588: error=0.05852466401040089
101 + 44 = 145 : 145.0
Epoch #9589: error=0.04565064002056134
27 + 10 = 37 : 37.0
Epoch #9590: error=0.05804361544039232
109 + 39 = 148 : 148.0
Epoch #9591: error=0.05850520309323492
45 + 44 = 89 : 89.0
Epoch #9592: error=0.07038472743015227
41 + 93 = 134 : 134.0
Epoch #9593: error=0.05510974508920766
123 + 125 = 248 : 248.0
Epoch #9594: error=0.044780347954383196
5 + 2 = 7 : 7.0
Epoch #9595: error=0.0648378730804179
48 + 18 = 66 : 66.0
Epoch #9596: error=0.057736328788412075
100 + 102 = 202 : 202.0
Epoch #9597: error=0.07693026924654739
95 + 48 = 143 : 143.0
Epoch #9598: error=0.05297106130288292
55 + 61 = 116 : 116.0
Epoch #9599: error=0.06919695374169522
8 + 86 = 94 : 94.0
Epoch #9600: error=0.0439703484502182
38 + 19 = 57 : 57.0
Epoch #9601: error=0.056851255189001776
122 + 87 = 209 : 209.0
Epoch #9602: error=0.06341855476031

26 + 107 = 133 : 133.0
Epoch #9703: error=0.04784118358320577
89 + 89 = 178 : 178.0
Epoch #9704: error=0.06286840469008512
57 + 62 = 119 : 119.0
Epoch #9705: error=0.05402675224472712
6 + 15 = 21 : 21.0
Epoch #9706: error=0.06429074646312005
58 + 119 = 177 : 177.0
Epoch #9707: error=0.06809023143581854
46 + 94 = 140 : 140.0
Epoch #9708: error=0.0557661762076496
7 + 77 = 84 : 84.0
Epoch #9709: error=0.06363412700893892
87 + 108 = 195 : 195.0
Epoch #9710: error=0.046578243273298466
27 + 74 = 101 : 101.0
Epoch #9711: error=0.06148671555241453
58 + 36 = 94 : 94.0
Epoch #9712: error=0.07989550072990861
25 + 118 = 143 : 143.0
Epoch #9713: error=0.0820707597063271
78 + 97 = 175 : 175.0
Epoch #9714: error=0.07059498860610189
120 + 41 = 161 : 161.0
Epoch #9715: error=0.057471348092442254
55 + 103 = 158 : 158.0
Epoch #9716: error=0.09316374531571243
102 + 31 = 133 : 133.0
Epoch #9717: error=0.05627243402479887
121 + 83 = 204 : 204.0
Epoch #9718: error=0.06749150370421332
10 + 55 = 65 : 65.0
Epoc

Epoch #9822: error=0.09935622517802839
82 + 86 = 168 : 168.0
Epoch #9823: error=0.07550654451491604
20 + 126 = 146 : 146.0
Epoch #9824: error=0.09604152672139878
107 + 117 = 224 : 224.0
Epoch #9825: error=0.06423180045606253
46 + 60 = 106 : 106.0
Epoch #9826: error=0.06460501891171821
106 + 50 = 156 : 156.0
Epoch #9827: error=0.042977324275640535
66 + 71 = 137 : 137.0
Epoch #9828: error=0.06177269911534557
48 + 18 = 66 : 66.0
Epoch #9829: error=0.04140672618058071
77 + 65 = 142 : 142.0
Epoch #9830: error=0.06491595712048706
51 + 116 = 167 : 167.0
Epoch #9831: error=0.06241328668073369
96 + 67 = 163 : 163.0
Epoch #9832: error=0.06532422958868239
118 + 100 = 218 : 218.0
Epoch #9833: error=0.0713320330566559
48 + 112 = 160 : 160.0
Epoch #9834: error=0.0760006575672204
29 + 67 = 96 : 96.0
Epoch #9835: error=0.08179014101802928
69 + 27 = 96 : 96.0
Epoch #9836: error=0.06027862858206684
10 + 52 = 62 : 62.0
Epoch #9837: error=0.05895916065517748
110 + 6 = 116 : 116.0
Epoch #9838: error=0.0603

Epoch #9940: error=0.039999031173253465
73 + 19 = 92 : 92.0
Epoch #9941: error=0.08292057569116282
87 + 57 = 144 : 144.0
Epoch #9942: error=0.04588965929429009
55 + 52 = 107 : 107.0
Epoch #9943: error=0.05324234272024893
124 + 88 = 212 : 212.0
Epoch #9944: error=0.06062877000522335
116 + 89 = 205 : 205.0
Epoch #9945: error=0.07788233668982085
114 + 70 = 184 : 184.0
Epoch #9946: error=0.04859487828934431
51 + 35 = 86 : 86.0
Epoch #9947: error=0.10037517500766145
80 + 95 = 175 : 175.0
Epoch #9948: error=0.03533760262959077
33 + 5 = 38 : 38.0
Epoch #9949: error=0.05038357877500892
60 + 13 = 73 : 73.0
Epoch #9950: error=0.06262310469685192
11 + 107 = 118 : 118.0
Epoch #9951: error=0.06502991920139765
43 + 69 = 112 : 112.0
Epoch #9952: error=0.06236966528689383
74 + 82 = 156 : 156.0
Epoch #9953: error=0.1085984849050006
66 + 62 = 128 : 128.0
Epoch #9954: error=0.06107809511903583
119 + 16 = 135 : 135.0
Epoch #9955: error=0.05536782109823882
0 + 13 = 13 : 13.0
Epoch #9956: error=0.0586661551